In [105]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
from matplotlib import pyplot as plt
LCIS = pd.read_csv('C:\\Users\\skyamz\\Desktop\\ppdai_3_23\\LCIS.csv') # Loan Periodic，标的还款计划和还款记录表

In [106]:
LCIS01 = LCIS[LCIS['recorddate'].isin(['2016/12/31', '2017/2/28','2017/1/31', '2016/11/30', '2016/10/31', '2016/9/30'])]

In [107]:
#要计算累计收益率，必须要计算罚息，当月净增坏账，利息，当月待收本金和当月月初坏
# 计算 每月应还款额，每期本金，利息
LCIS01['每月还款额'] = (LCIS01['借款金额']*(LCIS01['借款利率']*0.01/12)*pow(1+ LCIS01['借款利率']*0.01/12,LCIS01['借款期限'])/(pow(1+ LCIS01['借款利率']*0.01/12,LCIS01['借款期限'])-1))/LCIS01['借款期限']
LCIS01['每月本金'] = (LCIS01['每月还款额'] - LCIS01['借款金额']*(LCIS01['借款利率']*0.01/12))*pow(1+ LCIS01['借款利率']*0.01/12,LCIS01['当前到期期数']-1)
LCIS01['每月利息'] = LCIS01['每月还款额'] - LCIS01['每月本金']

C:\Users\skyamz\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\skyamz\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\skyamz\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [108]:
#计算罚息
Penalty_interest_long = LCIS01[LCIS01['借款期限']>6]
Penalty_interest_short = LCIS01[LCIS01['借款期限']<=6]
Penalty_interest_long['罚息'] = (0.24/360)*Penalty_interest_long['每月本金']*Penalty_interest_long['标当前逾期天数']
Penalty_interest_short['罚息'] = (0.224/360)*Penalty_interest_short['每月本金']*Penalty_interest_short['标当前逾期天数']
Penalty_interest = pd.concat([Penalty_interest_long,Penalty_interest_short])

C:\Users\skyamz\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\skyamz\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [109]:
#计算坏账
bad_debt = Penalty_interest[Penalty_interest['标当前逾期天数']>90]
bad_debt['坏账'] = bad_debt['每月本金'] + bad_debt['每月利息']

C:\Users\skyamz\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [120]:
bad_debt['累计收益'] = bad_debt['每月利息']+ bad_debt['罚息'] - bad_debt['坏账']

C:\Users\skyamz\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [121]:
income['累计收益'] 

recorddate
2016/10/31   -25007.875869
2016/11/30   -21012.781207
2016/12/31   -13492.827609
2016/9/30    -29208.151738
2017/2/28     -2243.211048
Name: 累计收益, dtype: float64

In [122]:
bad_debt['累计收益率'] = pow(1+ (bad_debt['每月利息']+ bad_debt['罚息'] - bad_debt['坏账'])/(bad_debt['总待还本金']+ bad_debt['坏账']),12)-1

C:\Users\skyamz\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [123]:
def Tran_Date(v):
            if v == '2016/9/30':return 1609
            elif v == '2016/10/31':return 1610
            elif v == '2016/11/30':return 1611
            elif v == '2016/12/31':return 1612
            elif v == '2017/2/28' :return 1702
            else:return 0

In [124]:
bad_debt['newrecorddate'] = bad_debt['recorddate'].map(Tran_Date)

C:\Users\skyamz\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [135]:
income = bad_debt.groupby('newrecorddate')['累计收益','累计收益率'].sum()
income = income.reset_index()

In [136]:
income

,newrecorddate,累计收益,累计收益率
0,1609,-29208.151738,1.689273e+11
1,1610,-25007.875869,1.760231e+11
2,1611,-21012.781207,2.346169e+11
3,1612,-13492.827609,1.355482e+08
4,1702,-2243.211048,1.147765e+08


In [137]:
fig = plt.figure(figsize = (20,10))#设置大小
plt.title('6month-income')#标题
plt.xlabel('Month')#X轴
plt.ylabel('income')#Y轴

plt.plot(income['newrecorddate'],income['累计收益'],'',label='income')

In [138]:
plt.show()

In [144]:
fig = plt.figure(figsize = (20,10))#设置大小
plt.title('6month-income-rate')#标题
plt.xlabel('Month')#X轴
plt.ylabel('income-rate')#Y轴

plt.plot(income['newrecorddate'],income['累计收益率']/10000000000,'',label='income')

In [145]:
plt.show()